In [1]:
pip install onnxruntime


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install onnx

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
from torchvision import datasets

dataset = datasets.ImageFolder('/Users/Usuario/Desktop/sign detector - 1.1/dataset2/test')
print(dataset.class_to_idx)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


In [6]:
#PARA TESTEAR TODA LA CARPETA DE IMAGENES

import onnxruntime
import numpy as np
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F
import os
import torch
import time

# Ruta del modelo ONNX
model_path = "/users/Usuario/Desktop/sign detector - 1.1/onnx_model/stacked_model_mobilenet_resnet.onnx"
session = onnxruntime.InferenceSession(model_path)

# Transformación igual a la del entrenamiento
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Clases en orden
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
               'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'borrar', 'nada', 'espacio']
class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}

def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img_t = transform(img)
    return img_t.numpy()[np.newaxis, ...]

# Ruta a la carpeta de test
test_dir = ("/users/Usuario/Desktop/sign detector - 1.1/dataset2/test")

# Obtener todas las rutas de imágenes y etiquetas
image_paths = []
labels = []

for class_name in os.listdir(test_dir):
    class_path = os.path.join(test_dir, class_name)
    if not os.path.isdir(class_path) or class_name not in class_to_idx:
        continue
    label_idx = class_to_idx[class_name]

    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        image_paths.append(img_path)
        labels.append(label_idx)

total_images = len(image_paths)
correct = 0
losses = []

input_name = session.get_inputs()[0].name

start_time = time.time()

# Procesar cada imagen
for i, (img_path, label_idx) in enumerate(zip(image_paths, labels), 1):
    try:
        input_image = preprocess_image(img_path)
    except Exception as e:
        print(f"Error con imagen {img_path}: {e}")
        continue

    outputs = session.run(None, {input_name: input_image})
    logits = outputs[0]
    pred = np.argmax(logits, axis=1)[0]

    if pred == label_idx:
        correct += 1

    logits_tensor = torch.tensor(logits)
    target_tensor = torch.tensor([label_idx])
    loss = F.cross_entropy(logits_tensor, target_tensor)
    losses.append(loss.item())

    # Progreso y tiempo estimado
    elapsed = time.time() - start_time
    avg_time_per_image = elapsed / i
    remaining_time = avg_time_per_image * (total_images - i)

    print(f"[{i}/{total_images}] Procesando: {os.path.basename(img_path)} | "
          f"Tiempo transcurrido: {elapsed:.1f}s | "
          f"Tiempo restante estimado: {remaining_time:.1f}s", end='\r')

print("\n\nEvaluación completada.\n")

# Resultados finales
accuracy = correct / total_images * 100
avg_loss = np.mean(losses)

print(f"Total de imágenes evaluadas: {total_images}")
print(f"Precisión (accuracy): {accuracy:.2f}%")
print(f"Pérdida promedio (loss): {avg_loss:.4f}")

[2600/2600] Procesando: z_7_rotate_3.jpeg | Tiempo transcurrido: 648.5s | Tiempo restante estimado: 0.0sssss

Evaluación completada.

Total de imágenes evaluadas: 2600
Precisión (accuracy): 97.92%
Pérdida promedio (loss): 0.0694


In [9]:
import os
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 29
dataset_path = 'C:/users/Usuario/Desktop/sign detector - 1.1/dataset2'

# === Definición modelos base ===

# ResNet50
resnet = models.resnet50(weights='IMAGENET1K_V1')
in_features_resnet = resnet.fc.in_features
resnet.fc = nn.Sequential(
    nn.Linear(in_features_resnet, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
)
#resnet.load_state_dict(torch.load("resnet_best_model.pth", map_location=device))
resnet = resnet.to(device)

# MobileNetV2
mobilenet = models.mobilenet_v2(weights='IMAGENET1K_V1')
in_features_mobilenet = mobilenet.classifier[1].in_features
mobilenet.classifier = nn.Sequential(
    nn.Linear(in_features_mobilenet, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
)
#mobilenet.load_state_dict(torch.load("mobilenetv2_best_model.pth", map_location=device))
mobilenet = mobilenet.to(device)

# === Modelo Stacking ===
class StackedModel(nn.Module):
    def __init__(self, model1, model2, num_classes=29):
        super(StackedModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.classifier = nn.Sequential(
            nn.Linear(num_classes * 2, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        out1 = self.model1(x)
        out2 = self.model2(x)
        combined = torch.cat((out1, out2), dim=1)
        return self.classifier(combined)

stacked_model = StackedModel(resnet, mobilenet, num_classes=num_classes).to(device)

# === Congelar modelos base para entrenar solo la capa combinadora ===
for param in stacked_model.model1.parameters():
    param.requires_grad = False

for param in stacked_model.model2.parameters():
    param.requires_grad = False

# === Transforms, datasets y loaders ===
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = ImageFolder(os.path.join(dataset_path, 'train'), transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = ImageFolder(os.path.join(dataset_path, 'test'), transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# === Funciones de entrenamiento y evaluación ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, stacked_model.parameters()), lr=1e-4)

def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, running_corrects, total_samples = 0, 0, 0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == targets).item()
        total_samples += inputs.size(0)
    return running_loss / total_samples, running_corrects / total_samples

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == targets).sum().item()
            total_samples += targets.size(0)

    avg_loss = total_loss / total_samples
    acc = total_correct / total_samples
    return avg_loss, acc

# === Entrenamiento ===
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(stacked_model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(stacked_model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

# === Evaluación final ===
test_loss, test_acc = evaluate(stacked_model, test_loader, criterion, device)
print(f"\n🔗 Test Acc Stacked Model: {test_acc:.4f} ({test_acc*100:.2f}%) | Test Loss: {test_loss:.4f}")

# === Exportar a ONNX ===
dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(
    stacked_model,
    dummy_input,
    "stacked_model_mobilenet_resnet.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)

print("✅ Modelo combinado exportado a stacked_model_mobilenet_resnet.onnx")


KeyboardInterrupt: 